In [2]:
import os
%pwd

'd:\\Data Science\\apple_disease_detection\\apple-disease-detection\\research'

In [3]:
os.chdir("d:\\Data Science\\apple_disease_detection\\apple-disease-detection")
%pwd

'd:\\Data Science\\apple_disease_detection\\apple-disease-detection'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from apple.constants import *
from src.apple.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config
    

In [10]:
import os
import zipfile
import urllib.request as request
from src.apple import logger
from apple.utils.common import create_directories, get_size





class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"downloaded {filename} with following info: \n{headers}")
        else:
            logger.info(f"File already exists with size {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-16 15:26:28,548: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-16 15:26:28,571: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-16 15:26:28,584: INFO: common: created directory at: artifacts]
[2024-12-16 15:26:28,590: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-16 15:26:42,250: INFO: 696519733: downloaded artifacts/data_ingestion/data.zip with following info: 
Connection: close
Content-Length: 5761987
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "beb89012f85825e2ea9cd2862dcc7ab6285d21dc2131c895831a56bafc6e1f39"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D949:1CC209:1BB176:351748:67600057
Accept-Ranges: bytes
Date: Mon, 16 Dec 2024 10:26:31 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990024-FJR
X-Cache: 